In [1]:
API_TOKEN = 'eyJtaXJvLm9yaWdpbiI6ImV1MDEifQ_uxjJtfs6yYlcL1XMLifDWVOSP2c'
BOARD_ID = 'uXjVK1N3F5U='

In [15]:
## Basic | Very nice and working


import requests
import pandas as pd

HEADERS = {
    'Authorization': f'Bearer {API_TOKEN}',
    'Content-Type': 'application/json'
}

# Function to create a batch of shapes (nodes) in Miro
def create_shapes(board_id, nodes):
    url = f'https://api.miro.com/v2/boards/{board_id}/shapes'
    responses = []
    print(f"Creating {len(nodes)} shapes...")
    for idx, node in enumerate(nodes):
        print("content;--", node['content'], type(node['content']))
        payload = {
            "data": {
                "content": node['content'],
                "shape": node['shape']
            },
            "style": {
                "borderColor": node['borderColor'],
                "borderOpacity": "1.0",
                "borderStyle": "normal",
                "borderWidth": "1",
                "color": "#1a1a1a",
                "fillColor": node['fillColor'],
                "fillOpacity": "1.0",
                "fontFamily": "arial",
                "fontSize": "10",
                "textAlign": "center",
                "textAlignVertical": "middle"
            },
            "position": {
                "x": node['x'],
                "y": node['y']
            },
            "geometry": {
                "height": 60,
                "rotation": 0,
                "width": 100
            }
        }
        response = requests.post(url, headers=HEADERS, json=payload)
        if response.status_code == 201:
            responses.append(response.json())
            print(f"Shape {idx+1}/{len(nodes)} created successfully: {node['content']}")
        else:
            print(f"Error creating shape {idx+1}/{len(nodes)}: {response.status_code}, {response.text}")
    return responses

# Function to create a batch of connectors (edges) in Miro
def create_connectors(board_id, connectors):
    url = f'https://api.miro.com/v2/boards/{board_id}/connectors'
    print(f"Creating {len(connectors)} connectors...")
    for idx, connector in enumerate(connectors):
        payload = {
            "startItem": {
                "id": connector['start_id'],
                # "position": {"x": "50%", "y": "0%"},
                "snapTo": "auto"
            },
            "endItem": {
                "id": connector['end_id'],
                # "position": {"x": "50%", "y": "0%"},
                "snapTo": "auto"
            },
            "shape": "straight",
            "style": {
                "color": "#9510ac",
                "endStrokeCap": "none",
                "fontSize": "15",
                "startStrokeCap": "none",
                "strokeColor": "#2d9bf0",
                "strokeStyle": "normal",
                "strokeWidth": "2.0",
                "textOrientation": "horizontal"
            }
        }
        response = requests.post(url, headers=HEADERS, json=payload)
        if response.status_code == 200:
            print(f"Connector {idx+1}/{len(connectors)} created successfully")
        else:
            print(f"Error creating connector {idx+1}/{len(connectors)}: {response.status_code}, {response.text}")
    return "Connectors created successfully"

# Create graph from DataFrame
def create_graph_in_miro(board_id, df):
    nodes = {}
    shapes_payload = []
    connectors_payload = []
    x, y = 0, 0
    x_step, y_step = 200, 100

    print("Preparing shapes and connectors...")
    for index, row in df.iterrows():
        referer = row['Referer']
        full_path = row['full_path']

        if referer not in nodes:
            node = {
                'content': referer,
                'shape': 'trapezoid',
                'borderColor': '#000000',
                'fillColor': '#ADD8E6',
                'x': x,
                'y': y
            }
            nodes[referer] = len(shapes_payload)
            shapes_payload.append(node)
            x += x_step

        if full_path not in nodes:
            node = {
                'content': full_path,
                'shape': 'rectangle',
                'borderColor': '#000000',
                'fillColor': '#ADD8E6',
                'x': x,
                'y': y
            }
            nodes[full_path] = len(shapes_payload)
            shapes_payload.append(node)
            y += y_step
    
    print("Shapes and connectors prepared. : ", len(shapes_payload))
    shapes_response = create_shapes(board_id, shapes_payload)

    print("Shape created successfully.")

    if shapes_response:
        print("starting for connector.")
        widget_ids = [widget['id'] for widget in shapes_response]
        for referer, full_path in zip(df['Referer'], df['full_path']):
            start_widget_id = widget_ids[nodes[referer]]
            end_widget_id = widget_ids[nodes[full_path]]
            connector = {
                'start_id': start_widget_id,
                'end_id': end_widget_id
            }
            connectors_payload.append(connector)
        print("connectors_payload len:-", len(connectors_payload))
        create_connectors(board_id, connectors_payload)

# Generate a random DataFrame with 500 nodes for demonstration
num_nodes = 10  # Adjust as needed for 500-1000 nodes

df = pd.read_json('data.json')
df['Referer'] = df['Referer'].fillna("None")

df["full_path"] = "https://"+df['Host']+df['path']
# Create the graph in Miro
create_graph_in_miro(BOARD_ID, df)


Preparing shapes and connectors...
Shapes and connectors prepared. :  137
Creating 137 shapes...
content;-- https://www.evernote.com/ <class 'str'>
Shape 1/137 created successfully: https://www.evernote.com/
content;-- https://api.evernote.com/command/v1/batch <class 'str'>
Shape 2/137 created successfully: https://api.evernote.com/command/v1/batch
content;-- https://web.desktop.evernote.com/ <class 'str'>
Shape 3/137 created successfully: https://web.desktop.evernote.com/
content;-- https://fonts.googleapis.com/css <class 'str'>
Shape 4/137 created successfully: https://fonts.googleapis.com/css
content;-- https://web.desktop.evernote.com/builds/next_20240626134412/ce/chunks/emoticons.76c5ab72c88d4ba7ec35.js <class 'str'>
Shape 5/137 created successfully: https://web.desktop.evernote.com/builds/next_20240626134412/ce/chunks/emoticons.76c5ab72c88d4ba7ec35.js
content;-- https://web.desktop.evernote.com/builds/next_20240626134412/ce/chunks/templatemanager.70a626ad43fa28842dd4.js <class 's

In [5]:
a = [1,2,4,5,7,5,3]
a[4:]

[7, 5, 3]

In [27]:
import requests
import pandas as pd

# Replace with your Miro API token and board ID


HEADERS = {
    'Authorization': f'Bearer {API_TOKEN}',
    'Content-Type': 'application/json'
}

# Function to create a batch of shapes (nodes) in Miro
def create_shapes(board_id, nodes):
    url = f'https://api.miro.com/v2/boards/{board_id}/shapes'
    responses = []
    print(f"Creating {len(nodes)} shapes...")
    for idx, node in enumerate(nodes):
        payload = {
            "data": {
                "content": node['content'],
                "shape": node['shape']
            },
            "style": {
                "borderColor": node['borderColor'],
                "borderOpacity": "1.0",
                "borderStyle": "normal",
                "borderWidth": "1",
                "color": "#1a1a1a",
                "fillColor": node['fillColor'],
                "fillOpacity": "1.0",
                "fontFamily": "arial",
                "fontSize": "10",
                "textAlign": "center",
                "textAlignVertical": "middle"
            },
            "position": {
                "x": node['x'],
                "y": node['y']
            },
            "geometry": {
                "height": 60,
                "rotation": 0,
                "width": 100
            }
        }
        response = requests.post(url, headers=HEADERS, json=payload)
        if response.status_code == 201:
            responses.append(response.json())
            print(f"Shape {idx+1}/{len(nodes)} created successfully: {node['content']}")
        else:
            print(f"Error creating shape {idx+1}/{len(nodes)}: {response.status_code}, {response.text}")
    return responses

# Function to create a batch of connectors (edges) in Miro
def create_connectors(board_id, connectors):
    url = f'https://api.miro.com/v2/boards/{board_id}/connectors'
    print(f"Creating {len(connectors)} connectors...")
    for idx, connector in enumerate(connectors):
        payload = {
            "startItem": {
                "id": connector['start_id'],
                # "position": {"x": "50%", "y": "0%"},
                "snapTo": "auto"
            },
            "endItem": {
                "id": connector['end_id'],
                # "position": {"x": "50%", "y": "0%"},
                "snapTo": "auto"
            },
            "shape": "straight",
            "style": {
                "color": "#9510ac",
                "endStrokeCap": "none",
                "fontSize": "15",
                "startStrokeCap": "none",
                "strokeColor": "#2d9bf0",
                "strokeStyle": "normal",
                "strokeWidth": "2.0",
                "textOrientation": "horizontal"
            }
        }
        response = requests.post(url, headers=HEADERS, json=payload)
        if response.status_code == 200:
            print(f"Connector {idx+1}/{len(connectors)} created successfully")
        else:
            print(f"Error creating connector {idx+1}/{len(connectors)}: {response.status_code}, {response.text}")
    return "Connectors created successfully"

# Create graph from DataFrame
def create_mind_map_in_miro(board_id, df):
    nodes = {}
    shapes_payload = []
    connectors_payload = []
    x, y = 500, 500
    x_step, y_step = 300, 200

    print("Preparing shapes and connectors for mind map...")
    for index, row in df.iterrows():
        central_topic = row['Central Topic']
        subtopic = row['Subtopic']

        if central_topic not in nodes:
            node = {
                'content': central_topic,
                'shape': 'circle',
                'borderColor': '#000000',
                'fillColor': '#ADD8E6',
                'x': x,
                'y': y
            }
            shapes_response = create_shapes(board_id, [node])
            if shapes_response:
                node['id'] = shapes_response[0]['id']
                nodes[central_topic] = node['id']
                shapes_payload.append(node)
                y += y_step
            else:
                print(f"Error creating shape for {central_topic}")

        if subtopic not in nodes:
            node = {
                'content': subtopic,
                'shape': 'rectangle',
                'borderColor': '#000000',
                'fillColor': '#ADD8E6',
                'x': x + x_step,
                'y': y - y_step
            }
            shapes_response = create_shapes(board_id, [node])
            if shapes_response:
                node['id'] = shapes_response[0]['id']
                nodes[subtopic] = node['id']
                shapes_payload.append(node)
                y += y_step
            else:
                print(f"Error creating shape for {subtopic}")

        connector = {
            'start_id': nodes[central_topic],
            'end_id': nodes[subtopic]
        }
        connectors_payload.append(connector)

    print("Shapes and connectors prepared for mind map.")
    create_connectors(board_id, connectors_payload)

# Generate a more complex DataFrame with 20 central topics and 60 subtopics for demonstration
num_topics = 2
num_subtopics_per_topic = 3
central_topics = [f'Topic_{i}' for i in range(num_topics)]
subtopics = []

# Create subtopics for each central topic
for topic in central_topics:
    subtopics.extend([(topic, f'Subtopic_{i}') for i in range(num_subtopics_per_topic)])

# Convert to DataFrame
data = {
    'Central Topic': [x[0] for x in subtopics],
    'Subtopic': [x[1] for x in subtopics]
}
df = pd.DataFrame(data)

# Create the mind map in Miro
create_mind_map_in_miro(BOARD_ID, df)


Preparing shapes and connectors for mind map...
Creating 1 shapes...
Shape 1/1 created successfully: Topic_0
Creating 1 shapes...
Shape 1/1 created successfully: Subtopic_0
Creating 1 shapes...
Shape 1/1 created successfully: Subtopic_1
Creating 1 shapes...
Shape 1/1 created successfully: Subtopic_2
Creating 1 shapes...
Shape 1/1 created successfully: Topic_1
Shapes and connectors prepared for mind map.
Creating 6 connectors...
Connector 1/6 created successfully
Connector 2/6 created successfully
Connector 3/6 created successfully
Connector 4/6 created successfully
Connector 5/6 created successfully
Connector 6/6 created successfully


In [28]:
df

,Central Topic,Subtopic
0,Topic_0,Subtopic_0
1,Topic_0,Subtopic_1
2,Topic_0,Subtopic_2
3,Topic_1,Subtopic_0
4,Topic_1,Subtopic_1
5,Topic_1,Subtopic_2
